In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
cutterDL = pd.read_csv('DraftLeagueCutter.csv')
cutterMLB = pd.read_csv('MLBCutter.csv')

In [3]:
cutterDL['league'] = 'Draft League'
cutterMLB['league'] = 'MLB'

In [4]:
cutterDL = cutterDL.astype(str)
cutterMLB = cutterMLB.astype(str)

In [5]:
pitchersDL = list(set(cutterDL['pitcher']))

In [6]:
cutter = pd.concat([cutterDL, cutterMLB])

In [7]:
def combined_features(row):
    return row['avgvelo']+" "+row['avgspinrate']+" "+row['avghorzbreak']+" "+row['avgvertbreak']+" "+row['avgeffectvelo']

In [8]:
#fig, ax = plt.subplots(figsize=(100,100))
#sns.heatmap(cosine_sim,cmap='magma',ax=ax)

In [9]:
def get_index_from_pitcher(pitcher):
    return cutter[cutter.pitcher == pitcher].index[0]

In [10]:
def get_pitcher_from_index(index):
    return cutter[cutter.index == index].values[0][0]

In [11]:
mlb_draft_pitcher = []
mlb_comp = []
cos_value = []

for x in pitchersDL:
    
    mlb_draft_pitcher.append(x)
    
    try:

        cutter = pd.concat([cutterDL, cutterMLB])

        cutter["combined_features"] = cutter.apply(combined_features, axis =1)

        cv = CountVectorizer()
        count_matrix = cv.fit_transform(cutter["combined_features"])

        cosine_sim = cosine_similarity(count_matrix)

        pitcher_like = x
        cutter = cutter[(cutter['pitcher'] == pitcher_like) | (cutter['league'] == 'MLB')]

        player_index = get_index_from_pitcher(pitcher_like)

        similar_pitchers = list(enumerate(cosine_sim[player_index]))

        sorted_similar_pitchers = sorted(similar_pitchers, key=lambda x:x[1], reverse=True)

        mlb_comp.append(get_pitcher_from_index(sorted_similar_pitchers[1][0]))
        
        cos_value.append(sorted_similar_pitchers[1][1])
                
    except IndexError:
        mlb_comp.append('No Comp')
        cos_value.append('No Similarity')
        pass

In [12]:
cosine_comp = pd.DataFrame(list(zip(mlb_draft_pitcher, mlb_comp, cos_value)), columns=['MLB DL Pitcher', 'MLB Comp', "Cosine Value"])
cosine_comp

,MLB DL Pitcher,MLB Comp,Cosine Value
0,"Parisi, Jack","Flexen, Chris",0.298142
1,"Chiriboga, Chris","Sampson, Adrian",0.455842
2,"Sapp, Eric","Baker, Bryan",0.235702
3,"Pickell, Cannon","González, Chi Chi",0.381385
4,"Olson, Mason","Arihara, Kohei",0.316228
5,"Modugno, John","Arihara, Kohei",0.316228
6,"Gonzalez, Joan","Sánchez, Sixto",0.333333
7,"Busse, Terry","Sampson, Adrian",0.455842
8,"Kane, Thomas","Rusin, Chris",0.251976
9,"Wiseman, Kevin","Sánchez, Sixto",0.333333


In [13]:
cosine_comp.to_csv('cutter_cosine_comp.csv', sep='\t')